Author: Joe  Krinke

Notebook Description: An implementation of ResNet50 for BME 590: Deep Learning in Healthcare.

In [ ]:
import keras
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds 
import keras.backend as K
import tensorflow as tf

K.set_image_data_format('channels_last')


Begin by defining the identity and convolutional blocks. These two pieces are the building blocks of ResNet50. 



In [ ]:
def identity_block(x, filters, mid_kernel_size):
    """ 
    The ResNet 50 Identity Block 

    Arguments:
    x: the input tensor
    filters: tuple of three filter sizes, one for each convolutional layer 
    mid_kernel_size: tuple of the kernel size of the middle convolutional layer

    Returns: 
    x: the tensor output of the identity block
    """
    # Get filter dimensions from input tuple

    (filter1, filter2, filter3) = filters

    # Obtain the initial value of x

    x_skip = x

    x = layers.Conv2D(filters=filter1, kernel_size=(1, 1), 
                      strides=(1,1), padding='valid', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=filter2, kernel_size=mid_kernel_size,
                      strides=(1, 1), padding='same', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=filter3, kernel_size=(1, 1), 
                      strides=(1,1), padding='valid', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    # Add skipped x to the other x data

    x = layers.Add()([x, x_skip])
    x = layers.Activation('relu')(x)

    return x


def conv_block(x, filters, mid_kernel_size, strides):
    """ 
    The ResNet 50 Convolutional Block

    Arguments:
    x: the input tensor
    filters: a tuple of three filter sizes, one for each convolutional layer
    mid_kernel_size: tuple of the kernel size of the middle convolutional layer
    strides: a tuple of stride values of the first convolutional layer

    Returns: 
    x: the tensor output of the convolutional block
    """
    # Get filter dimensions from input tuple

    (filter1, filter2, filter3) = filters

    # Obtain the initial value of x

    x_skip = x

    x = layers.Conv2D(filters=filter1, kernel_size=(1, 1),
                      strides=strides, padding='valid', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=filter2, kernel_size=mid_kernel_size,
                      strides=(1, 1), padding='same', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=filter3, kernel_size=(1, 1), 
                      strides=(1,1), padding='valid', use_bias = False)(x)
    x = layers.BatchNormalization(axis=3)(x)
    x = layers.Activation('relu')(x)


    # Make sure the dimensions of the "skip" match so we can add it to x

    x_skip = layers.Conv2D(filters = filter3, kernel_size = (1, 1), 
                           strides = strides, padding = 'valid', 
                           use_bias = False)(x_skip)
    x_skip = layers.BatchNormalization(axis = 3)(x_skip)

    # Add the "skipped" x to the other x data

    x = layers.Add()([x, x_skip])
    x = layers.Activation('relu')(x)

    return x

In [ ]:
def ResNet50(input_shape, num_classes):
  """ 
  Implementation of the ResNet50 model in Keras

  Arguments:
  input_shape: the shape of the image inputs 
  num_classes: the number of classes in the dataset

  Returns:
  model: a Keras Model
  """ 
  # Create the inputs using the input shape argument

  inputs = keras.Input(input_shape)
  
  # Conv1

  x = layers.ZeroPadding2D((3,3))(inputs)
  x = layers.Conv2D(filters = 64, input_shape = input_shape, 
                    kernel_size = (7,7), strides = (2,2), padding = 'same', #I'm not sure about the padding here... I don't think you're supposed to have it but it fixes the shape issues with 32x32 images
                    use_bias = False)(x)
  x = layers.BatchNormalization(axis = 3)(x)
  x = layers.Activation('relu')(x)
  x = layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)

  # Conv2

  x = conv_block(x, (64, 64, 256), (3,3), strides = (1,1))
  x = identity_block(x, (64, 64, 256), (3,3))
  x = identity_block(x, (64, 64, 256), (3,3))

  # Conv3

  x = conv_block(x, (128, 128, 512), (3,3), strides = (2,2))
  x = identity_block(x, (128, 128, 512), (3,3))
  x = identity_block(x, (128, 128, 512), (3,3))
  x = identity_block(x, (128, 128, 512), (3,3))

  # Conv4

  x = conv_block(x, (256, 256, 1024), (3,3), strides = (2,2))
  x = identity_block(x, (256, 256, 1024), (3,3))
  x = identity_block(x, (256, 256, 1024), (3,3))
  x = identity_block(x, (256, 256, 1024), (3,3))
  x = identity_block(x, (256, 256, 1024), (3,3))
  x = identity_block(x, (256, 256, 1024), (3,3))

  # Conv5

  x = conv_block(x, (512, 512, 2048), (3,3), strides = (2,2))
  x = identity_block(x, (512, 512, 2048), (3,3))
  x = identity_block(x, (512, 512, 2048), (3,3))

  # Average Pool

  x = keras.layers.AveragePooling2D(pool_size = (2,2))(x)

  # Output

  x = layers.Flatten()(x)
  x = layers.Dense(num_classes, activation = 'softmax')(x)

  # Create model
  
  model = keras.Model(inputs = inputs, outputs = x)
  
  return model

Now let's train the model on the CIFAR-10 data. This data consists of 60000 32x32 color images with 10 classes. I may be overfitting a bit based on the difference between the train and test sets.

In [ ]:
# Load in cifar10 data
# batch_size=-1 to get the full dataset in NumPy arrays from the returned tf.Tensor object

cifar10_train = tfds.load(name="cifar10", split=tfds.Split.TRAIN, batch_size= -1) 
cifar10_test = tfds.load(name="cifar10", split=tfds.Split.TEST, batch_size= -1)

# tfds.as_numpy: return a generator that yields NumPy array records out of a tf.data.Dataset

cifar10_train = tfds.as_numpy(cifar10_train) 
cifar10_test = tfds.as_numpy(cifar10_test)

# Get the x and y values

x_train, y_train = cifar10_train["image"], cifar10_train["label"] 
x_test, y_test = cifar10_test["image"], cifar10_test["label"]

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


KeyboardInterrupt: ignored

In [ ]:
# Create a model and compile it

resnet = ResNet50((32,32,3), 10)
resnet.compile(optimizer=keras.optimizers.SGD(learning_rate=.10, momentum = .90),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# Fit the resnet50 model and train

resnet.fit(x_train, y_train, epochs = 100, batch_size = 256)

In [ ]:
#Evaluate the model's performance on the test set
results = resnet.evaluate(x_test, y_test)

Now that we've tried the model on the CIFAR-10 data, we can try tackling a more complex dataset: CIFAR-100. This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class.

In [ ]:
# Load in cifar100 data
# batch_size=-1 to get the full dataset in NumPy arrays from the returned tf.Tensor object

cifar100_train = tfds.load(name="cifar100", split=tfds.Split.TRAIN, batch_size= -1) 
cifar100_test = tfds.load(name="cifar100", split=tfds.Split.TEST, batch_size= -1)

# tfds.as_numpy: return a generator that yields NumPy array records out of a tf.data.Dataset

cifar100_train = tfds.as_numpy(cifar100_train) 
cifar100_test = tfds.as_numpy(cifar100_test)

# Get the x and y

x_train_100, y_train_100 = cifar100_train["image"], cifar100_train["label"]
x_test_100, y_test_100 = cifar100_test["image"], cifar100_test["label"]

In [ ]:
# Create a model and compile it

resnet_100 = ResNet50((32,32,3), 100) # Change the number of classes from 10 to 100
resnet_100.compile(optimizer=keras.optimizers.SGD(learning_rate=.1, momentum=.90),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
# Fit the resnet50 model and train. The 100 just denotes the differing class number. 

resnet_100.fit(x_train_100, y_train_100, epochs = 100, batch_size = 256)

In [ ]:
# Evaluate the performance of the model on the test set

results = resnet_100.evaluate(x_test_100, y_test_100)

**Sources:**

ResNet Paper: https://arxiv.org/abs/1512.03385

Simple Architecture Description: https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33